In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
data = pd.read_csv('archive\loan_data_set.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [5]:
data['Gender'].fillna(data['Gender'].mode()[0],inplace=True)
data['Married'] = data['Married'].replace(np.NaN, data['Married'].mode()[0])
data['Dependents'].fillna(data['Dependents'].mode()[0],inplace=True)
data['Self_Employed'] = data['Self_Employed'].replace(np.NaN, data['Self_Employed'].mode()[0])

In [6]:
data['LoanAmount'].fillna(int(data['LoanAmount'].mean()),inplace=True)
data['Loan_Amount_Term'].fillna(int(data['Loan_Amount_Term'].mean()),inplace=True)
data['Credit_History'].fillna(data['Credit_History'].mode()[0],inplace=True)

In [7]:
data.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [8]:
dummies1=pd.get_dummies(data['Gender'])
dummies2=pd.get_dummies(data['Married'])
dummies2=dummies2.rename(columns = {'Yes':'Married_yes','No':'NotMarried'})
dummies3=pd.get_dummies(data['Dependents'])
dummies3=dummies3.rename(columns = {'0': 'Dependents_0','1': 'Dependents_1','2': 'Dependents_2','3+': 'Dependents_3+'})
dummies4=pd.get_dummies(data['Education'])
dummies5=pd.get_dummies(data['Self_Employed'])
dummies5=dummies5.rename(columns = {'Yes':'Self_Employed_yes','No':'NotSelf_Employed'})
dummies6=pd.get_dummies(data['Property_Area'])

In [9]:
data= pd.concat([data,dummies1],axis=1)
data= pd.concat([data,dummies2],axis=1)
data= pd.concat([data,dummies3],axis=1)
data= pd.concat([data,dummies4],axis=1)
data= pd.concat([data,dummies5],axis=1)
data= pd.concat([data,dummies6],axis=1)

In [10]:
data=data.drop(['Loan_ID','Gender','Married','Property_Area','Dependents','Self_Employed','Education','NotMarried','NotSelf_Employed','Not Graduate','Female'] , axis=1)


In [11]:
data['Loan_Status']= data['Loan_Status'].replace(['Y'],1)
data['Loan_Status']= data['Loan_Status'].replace(['N'],0)

In [12]:
data.head(10)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Male,Married_yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Graduate,Self_Employed_yes,Rural,Semiurban,Urban
0,5849,0.0,146.0,360.0,1.0,1,1,0,1,0,0,0,1,0,0,0,1
1,4583,1508.0,128.0,360.0,1.0,0,1,1,0,1,0,0,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,1,1,0,0,0,1,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,1,1,1,1,0,0,0,0,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,1,1,0,1,0,0,0,1,0,0,0,1
5,5417,4196.0,267.0,360.0,1.0,1,1,1,0,0,1,0,1,1,0,0,1
6,2333,1516.0,95.0,360.0,1.0,1,1,1,1,0,0,0,0,0,0,0,1
7,3036,2504.0,158.0,360.0,0.0,0,1,1,0,0,0,1,1,0,0,1,0
8,4006,1526.0,168.0,360.0,1.0,1,1,1,0,0,1,0,1,0,0,0,1
9,12841,10968.0,349.0,360.0,1.0,0,1,1,0,1,0,0,1,0,0,1,0


In [13]:
X=data.drop(columns='Loan_Status')
Y=pd.DataFrame(data['Loan_Status'])

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15,random_state=3)

In [15]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ApplicantIncome    614 non-null    int64  
 1   CoapplicantIncome  614 non-null    float64
 2   LoanAmount         614 non-null    float64
 3   Loan_Amount_Term   614 non-null    float64
 4   Credit_History     614 non-null    float64
 5   Loan_Status        614 non-null    int64  
 6   Male               614 non-null    uint8  
 7   Married_yes        614 non-null    uint8  
 8   Dependents_0       614 non-null    uint8  
 9   Dependents_1       614 non-null    uint8  
 10  Dependents_2       614 non-null    uint8  
 11  Dependents_3+      614 non-null    uint8  
 12  Graduate           614 non-null    uint8  
 13  Self_Employed_yes  614 non-null    uint8  
 14  Rural              614 non-null    uint8  
 15  Semiurban          614 non-null    uint8  
 16  Urban              614 non

In [16]:
data.shape

(614, 17)

In [17]:
clf = RandomForestClassifier(n_estimators=1000,max_features=15,max_depth=5,bootstrap = True)
clf.fit(X_train,Y_train)
predictions = clf.predict(X_test)
accuracyScores = accuracy_score(predictions, Y_test)

import pickle
file_name = 'Random_Forest.sav'
pickle.dump(clf, open(file_name,'wb'))


C:\Users\Thomas\AppData\Local\Temp\ipykernel_12804\718703694.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,Y_train)
